# RAG

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. 문서 로드 

In [10]:
file_path = '../data/Sustainability_report_2024_kr.pdf'

print('문서 로드')

loader = PyPDFLoader(file_path)
docs = loader.load()

문서 로드


In [11]:
len(docs)

83

In [14]:
docs[1].metadata

{'producer': 'Adobe PDF Library 15.0',
 'creator': 'Adobe InDesign 15.1 (Macintosh)',
 'creationdate': '2024-11-25T11:10:32+09:00',
 'moddate': '2024-11-25T11:10:46+09:00',
 'trapped': '/False',
 'source': '../data/Sustainability_report_2024_kr.pdf',
 'total_pages': 83,
 'page': 1,
 'page_label': '2'}

# 2. 텍스트 청킹

In [17]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    #separators=['\n\n', 'n', ' ', ''] 내부 code로 설정되어 있음
)

chunks = splitter.split_documents(docs)
print(len(chunks))

207


In [21]:
chunks[3]

Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2024-11-25T11:10:32+09:00', 'moddate': '2024-11-25T11:10:46+09:00', 'trapped': '/False', 'source': '../data/Sustainability_report_2024_kr.pdf', 'total_pages': 83, 'page': 3, 'page_label': '4'}, page_content='삼성전자 지속가능경영보고서 2024\n04\nOur Company AppendixMateriality Assessment Facts & Figures PrinciplePlanet People\nCEO 메시지\nMessage from \nOur CEO\n주주, 고객, 협력회사, 그리고 임직원 여러분,\n2023년은 고금리와 인플레이션, 지정학적 이슈 등 매우 불확실한 \n거시경제 환경과 함께, 메모리 산업 부진과 다양한 제품군에서의 경쟁 \n심화로 삼성전자에게 매우 어려운 한 해였습니다. 이토록 대내외적으로 \n어려운 환경에서도 지속 성장의 기반 마련을 위해 역대 최고 수준의 28.3\n조원을 연구개발에 투자하고, 53.1조원 수준의 전략적 시설투자로 기술 \n리더십을 강화하며 중장기 수요에 미리 대응할 수 있었던 것은 삼성전자를 \n아껴주시는 이해관계자 여러분의 관심과 격려 덕분입니다. 다시 한 번 \n깊이 감사 드립니다.\n급격한 변화를 겪고 있는 경제 상황에 맞춰, 기업의 지속가능경영 \n분야에서도 많은 변화가 일어나고 있습니다. 특히 기업의 지속가능경영 \n활동 정보 공개는 글로벌 비재무정보 공시 제도의 확산에 맞춰, 새로운 \n국면을 맞고 있습니다. 국제회계기준재단(IFRS Foundation)이 2023년 \n6월 지속가능성 지표를 확정한 것을 시작으로, EU의 지속가능성 보

In [19]:
chunks[3].page_content

'삼성전자 지속가능경영보고서 2024\n04\nOur Company AppendixMateriality Assessment Facts & Figures PrinciplePlanet People\nCEO 메시지\nMessage from \nOur CEO\n주주, 고객, 협력회사, 그리고 임직원 여러분,\n2023년은 고금리와 인플레이션, 지정학적 이슈 등 매우 불확실한 \n거시경제 환경과 함께, 메모리 산업 부진과 다양한 제품군에서의 경쟁 \n심화로 삼성전자에게 매우 어려운 한 해였습니다. 이토록 대내외적으로 \n어려운 환경에서도 지속 성장의 기반 마련을 위해 역대 최고 수준의 28.3\n조원을 연구개발에 투자하고, 53.1조원 수준의 전략적 시설투자로 기술 \n리더십을 강화하며 중장기 수요에 미리 대응할 수 있었던 것은 삼성전자를 \n아껴주시는 이해관계자 여러분의 관심과 격려 덕분입니다. 다시 한 번 \n깊이 감사 드립니다.\n급격한 변화를 겪고 있는 경제 상황에 맞춰, 기업의 지속가능경영 \n분야에서도 많은 변화가 일어나고 있습니다. 특히 기업의 지속가능경영 \n활동 정보 공개는 글로벌 비재무정보 공시 제도의 확산에 맞춰, 새로운 \n국면을 맞고 있습니다. 국제회계기준재단(IFRS Foundation)이 2023년 \n6월 지속가능성 지표를 확정한 것을 시작으로, EU의 지속가능성 보고지침\n(CSRD)과 미국 증권거래위원회(SEC) 기후공시 기준 역시 세부 내용을 \n순차적으로 확정하며 ESG 정보의 의무 공시 시대가 열리고 있습니다.\n이와 함께, EU 탄소국경조정제도(CBAM)와 EU 배터리규제 등을 통한 \n환경규제 역시 지속 강화되는 추세이고, 독일에서는 공급망의 인권과 \n근로환경 관리를 의무화하는 공급망실사법이 2023년 발효되었으며, \n2024년 5월 EU 공급망 실사지침(CSDDD)이 확정되는 등 인권 분야에 \n대한 관심 또한 지속 고조되고 있습니다. \n삼성전자는 이러한 추세에 맞춰 지속가능한 미래를 위한 노력을 계속해'

In [24]:
chunks[3].metadata

{'producer': 'Adobe PDF Library 15.0',
 'creator': 'Adobe InDesign 15.1 (Macintosh)',
 'creationdate': '2024-11-25T11:10:32+09:00',
 'moddate': '2024-11-25T11:10:46+09:00',
 'trapped': '/False',
 'source': '../data/Sustainability_report_2024_kr.pdf',
 'total_pages': 83,
 'page': 3,
 'page_label': '4'}

# 3. 임베딩 생성 Chroma DB 저장

In [ ]:
db_path = '../vectorstore/chromadb_rag_basic'

# load할때도 같은 모델 사용해야 함
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory=db_path,
    collection_name='samsung_2024'
)

# 4. 검색기 구성(retriever)

In [31]:
retriever = vector_store.as_retriever()
result = retriever.invoke('이 보고서에서 2024년 주요 지속 가능 목표가 뭘까?')
result

[Document(metadata={'total_pages': 83, 'creationdate': '2024-11-25T11:10:32+09:00', 'trapped': '/False', 'source': '../data/Sustainability_report_2024_kr.pdf', 'page': 3, 'moddate': '2024-11-25T11:10:46+09:00', 'producer': 'Adobe PDF Library 15.0', 'page_label': '4', 'creator': 'Adobe InDesign 15.1 (Macintosh)'}, page_content='삼성전자 지속가능경영보고서 2024\n04\nOur Company AppendixMateriality Assessment Facts & Figures PrinciplePlanet People\nCEO 메시지\nMessage from \nOur CEO\n주주, 고객, 협력회사, 그리고 임직원 여러분,\n2023년은 고금리와 인플레이션, 지정학적 이슈 등 매우 불확실한 \n거시경제 환경과 함께, 메모리 산업 부진과 다양한 제품군에서의 경쟁 \n심화로 삼성전자에게 매우 어려운 한 해였습니다. 이토록 대내외적으로 \n어려운 환경에서도 지속 성장의 기반 마련을 위해 역대 최고 수준의 28.3\n조원을 연구개발에 투자하고, 53.1조원 수준의 전략적 시설투자로 기술 \n리더십을 강화하며 중장기 수요에 미리 대응할 수 있었던 것은 삼성전자를 \n아껴주시는 이해관계자 여러분의 관심과 격려 덕분입니다. 다시 한 번 \n깊이 감사 드립니다.\n급격한 변화를 겪고 있는 경제 상황에 맞춰, 기업의 지속가능경영 \n분야에서도 많은 변화가 일어나고 있습니다. 특히 기업의 지속가능경영 \n활동 정보 공개는 글로벌 비재무정보 공시 제도의 확산에 맞춰, 새로운 \n국면을 맞고 있습니다. 국제회계기준재단(IFRS Foundation)이 2023년 \n6월 지속가능성 지표를 확정한 것을 시작으로, EU의 지속가능성 

In [35]:
for r in result:
    print(r.page_content[:30])
    print('=' * 100)

삼성전자 지속가능경영보고서 2024
04
Our Com
A Journey Towards  
a Sustaina
삼성전자 지속가능경영보고서 2024
13
Our Com
기회요인으로 예상하며, 기온 상승  및 수자원 고갈  


# 5. LANGCHAIN 연결

In [ ]:
# basic langchain

# 1. 프롬프트 
rag_prompt = ChatPromptTemplate.from_messages([
    ('system', """
    주어진 context만 근거로 간결하고 정확하게 대답해줘
    context에 없으면 문서에 '근거 없음'이라고 대답
    
    # context
    {context}
    """),
    ('human', '{question}')
])

# 2. 모델 선택
model = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0
)

#3. output_parser 선택

output_parser = StrOutputParser()

chain = rag_prompt | model | output_parser

In [ ]:
def format_docs_join(docs):
    
    tmp_docs = []
    
    for doc in docs:
        tmp_docs.append(doc.page_content)
    
    return '\n\n\---\n\n'.join(tmp_docs)

In [40]:
rag_chain = (
    {
    'context': retriever | RunnableLambda(format_docs_join),
    'question' : RunnablePassthrough()
    }
    | chain
)

In [42]:
result = rag_chain.invoke('이 보고서에서 2024년 주요 지속 가능 목표가 뭘까?')
print(result)

2024년 주요 지속 가능 목표는 전년 에너지 사용량 대비 10% 감축 목표 설정, 국내 업무용 차량을 100% 무공해차(전기/수소차)로 전환 계획, 온실가스 직접배출 감축을 위한 고효율 설비 교체 및 제조공정 효율화, 에너지 절감 활동, 그리고 외부 온실가스 감축사업 발굴 및 추진입니다.


# langchain + rag practice

In [72]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub

In [50]:
loader = PDFPlumberLoader('../data/Sustainability_report_2024_kr.pdf')
docs = loader.load()
docs[0].page_content

'삼성전자 지속가능경영보고서 2024\nA Journey Towards\na Sustainable Future\n'

In [86]:
def format_docs_join(docs):
    
    tmp_docs = []
    
    for doc in docs:
        tmp_docs.append(doc.page_content)
    
    return '\n\n\---\n\n'.join(tmp_docs)

In [49]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)
chunks[0].page_content

'삼성전자 지속가능경영보고서 2024\nA Journey Towards\na Sustainable Future'

In [52]:
openai_embedding = OpenAIEmbeddings(model='text-embedding-3-small')

vector_store = FAISS.from_documents(
    documents=chunks,
    embedding=openai_embedding,
)

vector_store.save_local('../vectorstore/samsung_2024_faiss')

In [ ]:
retriever = vector_store.as_retriever()

# 그냥 문자열로 prompt를 사용하면 chain 적용이 안되니 template로 감싸주는 것 주의!
# context외에는 응답하지 않도록 하면, llm이 학습된 내용은 제외하고 응답 가능(rag data 기준)
prompt = ChatPromptTemplate.from_messages([
    ('system', """
    주어진 context만 근거로 간결하고 정확하게 대답해줘
    context에 없으면 문서에 '근거 없음'이라고 대답
    
    # context
    {context}
    """),
    ('human', '{question}')
])

prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="\n    주어진 context만 근거로 간결하고 정확하게 대답해줘\n    context에 없으면 문서에 '근거 없음'이라고 대답\n\n    # context\n    {context}\n    "), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [85]:
llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0)

In [87]:
chain = (
    {
    'context': retriever | format_docs_join, 
    'question': RunnablePassthrough()
    }
    | prompt
    | llm 
    | StrOutputParser()
)

In [88]:
result = chain.invoke('2024 중대성 목표는 뭐야')
print(result)

2024년 삼성전자의 중대성 목표는 기업 활동이 외부 환경에 미치는 영향과 해당 주제가 회사에 재무적으로 미치는 영향을 모두 고려하여 환경·사회 영향, 재무 위험 및 기회를 평가하고, 이를 바탕으로 중대 주제를 선정하여 지속가능경영에 중대하게 수렴하는 것입니다.
